<a href="https://colab.research.google.com/github/Youssef-ElBakry/Logistic-Regression-ML/blob/main/Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction
Logistic Regression

The dataset: The dataset used in this model is from kaggle. It is a set of data that holds 4 values about the physical appearance of a banknote per row:
- variance
- skewness
- curtosis
- entropy

 These variables can be used to predict whether a banknote is real or forged. This data was collected using an industrial camera and so while technically not a toy data set, it is fairly simple like a toy dataset.

More information about the dataset can be found here: https://www.kaggle.com/datasets/shanks0465/banknoteauthentication/data


The model used in following program is logistic regression. This model predicts a binary value, in this instance, real or forged based on a set of numerical values.



#Logistic Regression

In [ ]:
#Import libraries
import pandas as pd
import numpy as np

#Open Dataset
url = "https://raw.githubusercontent.com/Youssef-ElBakry/Logistic-Regression-ML/main/data_banknote_authentication.csv"
cols = ["Variance", "Skewness", "Curtosis", "Entropy", "Class"] #Adding headers to file as dataset has no headers
df = pd.read_csv(url, header=None, names=cols)
df["Class"] = df["Class"].astype(int)
print("classes:", df["Class"].value_counts().to_dict())
print("unique:", df["Class"].unique())
print(df)

#Define global values
trainingRatio = 0.8
LabelCol = "LABEL"
seed = 450 #Fixed seed for rng as it makes results reproducable


#Split data into training and testing
#80/20 used as there is no hyperparameter tuning and therefore no need for validation
rng = np.random.default_rng(seed)
test_I = []
train_I = []

#Shuffle and sort data entries into training and testing
for key, row in df.groupby("Class", sort="False"):
  index = row.index.to_numpy()
  rng.shuffle(index)
  trainingRows = int(round(len(index) * trainingRatio)) #Determine size of training set in each class. This allows for a more even split. (80% of the Authentic class and 80% of the forged class)
  train_I.extend(index[:trainingRows])
  test_I.extend(index[trainingRows:])

#Shuffle training and testing set again as they are currently grouped by class
train = df.loc[train_I].sample(frac=1, random_state=seed).reset_index(drop=True)
test = df.loc[test_I].sample(frac=1, random_state=seed).reset_index(drop=True)

FEATS = ["Variance", "Skewness", "Curtosis", "Entropy"]

X_train = train[FEATS].to_numpy(dtype=np.float64)
y_train = train["Class"].to_numpy(dtype=np.float64)
X_test  = test[FEATS].to_numpy(dtype=np.float64)
y_test  = test["Class"].to_numpy(dtype=np.float64)

#Standardise data using mean and standardisation to find z-scores
mu = X_train.mean(axis=0) #Calculate mean of each column
sd = X_train.std(axis=0) #Calculate standard deviation for each column

X_train_std = (X_train - mu) / sd #Turn each value in each column to a zscore
X_test_std = (X_test - mu) / sd

#Add a column of ones, to allow for intercept
def add_bias(X):
  return np.c_[np.ones((X.shape[0], 1)), X]  # shape: (N, D+1)

Xtr = add_bias(X_train_std)   # (n_train, 1+4)
Xte = add_bias(X_test_std)

#Sigmoid function
def sigmoid(z):
  #Cuts off z so that it's value its >500 or <-500 as those numbers to the power of e is an unnecessarily large number
  z = np.clip(z, -500, 500)
  return 1.0 / (1.0 + np.exp(-z))

def logloss_and_grad(W, X, y):
  #Get number of features in array
  N = X.shape[0]

  z = X @ W   #Matrix multiplication of array and weights
  p = sigmoid(z)    #Map each score to a point in the sigmoid graph

  # log loss (cross-entropy)
  eps = 1e-12   #avoid log(0) by setting adding a very small number
  loss = -np.mean(y*np.log(p + eps) + (1 - y)*np.log(1 - p + eps))

  # gradient of the loss w.r.t. W
  err  = p - y    #Difference between actual value and predicted value
  grad = (X.T @ err) / N  #Vector form for finding gradient

  return loss, grad

def fit_logreg(X, y, lr=0.1, epochs=2000, seed=450):
    rng = np.random.default_rng(seed)
    W = rng.normal(scale=0.01, size=X.shape[1])   # small random init, shape (D,)
    for t in range(epochs):
        loss, grad = logloss_and_grad(W, X, y)
        W -= lr * grad #Gradient decent
        if (t % 200 == 0 or t == epochs - 1):
            print(f"epoch {t:4d}  loss={loss:.4f}")
        if not np.isfinite(loss):
            raise RuntimeError("Loss exploded/NaN. Lower lr (e.g., 0.05 or 0.01).")
    return W

W = fit_logreg(Xtr, y_train, lr=0.1, epochs=2000, seed=450)

def predict_proba(X, W):
    return sigmoid(X @ W)

def predict_label(X, W, thresh=0.5):
    return (predict_proba(X, W) >= thresh).astype(int)

proba = predict_proba(Xte, W)
pred  = predict_label(Xte, W)

# metrics
tp = int(((pred == 1) & (y_test == 1)).sum())
tn = int(((pred == 0) & (y_test == 0)).sum())
fp = int(((pred == 1) & (y_test == 0)).sum())
fn = int(((pred == 0) & (y_test == 1)).sum())

acc  = (tp + tn) / (tp + tn + fp + fn) #% of true negatives and positives out of all the values
prec = tp / (tp + fp) if (tp + fp) else 0.0 #Number of true positives in all the true and false positives.
rec  = tp / (tp + fn) if (tp + fn) else 0.0 #%of true positives compared to all true positives and false negatives (All real positives)
f1   = 2 * prec * rec / (prec + rec) if (prec + rec) else 0.0

print(f"\nPositives [{tp:3}  {fp:3}]")
print(f"Negatives [{tn:3}  {fn:3}]")
print(f"\nTest Accuracy={acc:.3f} Recall={rec:.3f}  Precision={prec:.3f}  F1={f1:.3f}")

# inspect learned weights
feat_names = ["(bias)", "Variance", "Skewness", "Curtosis", "Entropy"]
for name, w in zip(feat_names, W):
    print(f"{name:>10s}: {w:+.4f}")

classes: {0: 762, 1: 610}
unique: [0 1]
      Variance  Skewness  Curtosis  Entropy  Class
0      3.62160   8.66610   -2.8073 -0.44699      0
1      4.54590   8.16740   -2.4586 -1.46210      0
2      3.86600  -2.63830    1.9242  0.10645      0
3      3.45660   9.52280   -4.0112 -3.59440      0
4      0.32924  -4.45520    4.5718 -0.98880      0
...        ...       ...       ...      ...    ...
1367   0.40614   1.34920   -1.4501 -0.55949      1
1368  -1.38870  -4.87730    6.4774  0.34179      1
1369  -3.75030 -13.45860   17.5932 -2.77710      1
1370  -3.56370  -8.38270   12.3930 -1.28230      1
1371  -2.54190  -0.65804    2.6842  1.19520      1

[1372 rows x 5 columns]

Positives [122    5]
Negatives [147    0]

Test Accuracy=0.982 Recall=1.000  Precision=0.961  F1=0.980
    (bias): -1.0178
  Variance: -4.2103
  Skewness: -3.8551
  Curtosis: -3.6189
   Entropy: +0.2054


The above model is very effective at differentiating between real and forged banknotes. With an accuracy of 98.2%, there is only a small amount of room for improvement.

The model tends to predict that a note is real more often than forged, with its recall of 1. This means that very single real banknote was correctly determined to be real. In a real scenario however, this would be bad. Under realistic circumstances, false positives are worse than false negatives as adding fake banknotes into circulation would have far more negatives consequences than throwing away real banknotes. This can be adjusted by changing the threshold from 0.5. In a following example, hyperparameter tuning will be used to adjust the threshold to find a more accurate value

Important notes:
While accuracy is high, it is a flawed measure. If one of the two outcomes are rare, and the model chooses the common one for all outcomes, the accuracy will appear to be high. For this reason, the precision and recall value exists.

#Logistic regression with L1

In [ ]:
#Import libraries
import pandas as pd
import numpy as np

#Open Dataset
url = "https://raw.githubusercontent.com/Youssef-ElBakry/Logistic-Regression-ML/main/data_banknote_authentication.csv"
cols = ["Variance", "Skewness", "Curtosis", "Entropy", "Class"] #Adding headers to file as dataset has no headers
df = pd.read_csv(url, header=None, names=cols)
df["Class"] = df["Class"].astype(int)
print("classes:", df["Class"].value_counts().to_dict())
print("unique:", df["Class"].unique())
print(df)

#Define global values
trainingRatio = 0.8
LabelCol = "LABEL"
seed = 450 #Fixed seed for rng as it makes results reproducable


#Split data into training and testing
#80/20 used as there is no hyperparameter tuning and therefore no need for validation
rng = np.random.default_rng(seed)
test_I = []
train_I = []

#Shuffle and sort data entries into training and testing
for key, row in df.groupby("Class", sort="False"):
  index = row.index.to_numpy()
  rng.shuffle(index)
  trainingRows = int(round(len(index) * trainingRatio)) #Determine size of training set in each class. This allows for a more even split. (80% of the Authentic class and 80% of the forged class)
  train_I.extend(index[:trainingRows])
  test_I.extend(index[trainingRows:])

#Shuffle training and testing set again as they are currently grouped by class
train = df.loc[train_I].sample(frac=1, random_state=seed).reset_index(drop=True)
test = df.loc[test_I].sample(frac=1, random_state=seed).reset_index(drop=True)

FEATS = ["Variance", "Skewness", "Curtosis", "Entropy"]

X_train = train[FEATS].to_numpy(dtype=np.float64)
y_train = train["Class"].to_numpy(dtype=np.float64)
X_test  = test[FEATS].to_numpy(dtype=np.float64)
y_test  = test["Class"].to_numpy(dtype=np.float64)

#Standardise data using mean and standardisation to find z-scores
mu = X_train.mean(axis=0) #Calculate mean of each column
sd = X_train.std(axis=0) #Calculate standard deviation for each column

X_train_std = (X_train - mu) / sd #Turn each value in each column to a zscore
X_test_std = (X_test - mu) / sd

#Add a column of ones, to allow for intercept
def add_bias(X):
  return np.c_[np.ones((X.shape[0], 1)), X]  # shape: (N, D+1)

Xtr = add_bias(X_train_std)   # (n_train, 1+4)
Xte = add_bias(X_test_std)

#Sigmoid function
def sigmoid(z):
  #Cuts off z so that it's value its >500 or <-500 as those numbers to the power of e is an unnecessarily large number
  z = np.clip(z, -500, 500)
  return 1.0 / (1.0 + np.exp(-z))

def logloss_and_grad(W, X, y):
  #Get number of features in array
  N = X.shape[0]

  z = X @ W   #Matrix multiplication of array and weights
  p = sigmoid(z)    #Map each score to a point in the sigmoid graph

  # log loss (cross-entropy)
  eps = 1e-12   #avoid log(0) by setting adding a very small number
  loss = -np.mean(y*np.log(p + eps) + (1 - y)*np.log(1 - p + eps))

  # gradient of the loss w.r.t. W
  err  = p - y    #Difference between actual value and predicted value
  grad = (X.T @ err) / N  #Vector form for finding gradient

  return loss, grad

def l1_loss_no_bias(W, lam):
  # Do NOT regularize the bias term
  return lam * np.sum(np.abs(W[1:]))

def soft_threshold(v, tau):
    # Prox operator for L1: shrink toward zero, possibly to exactly zero
    return np.sign(v) * np.maximum(np.abs(v) - tau, 0.0)

def fit_logreg_L1(X, y, lr=0.1, epochs=2000, lam=1e-3, seed=450):
    rng = np.random.default_rng(seed)
    W = rng.normal(scale=0.01, size=X.shape[1])

    for t in range(epochs):
        loss, grad = logloss_and_grad(W, X, y)

        #Gradient step on the smooth part
        W -= lr * grad

        # 3) Prox step for L1 on the non-bias weights
        W[1:] = soft_threshold(W[1:], lr * lam)

        if t % 200 == 0 or t == epochs - 1:
            total = loss + l1_loss_no_bias(W, lam)
            nnz = int(np.count_nonzero(np.abs(W[1:]) > 0))
            print(f"epoch {t:4d}  logloss={loss:.4f}  + L1={total:.4f}  nnz={nnz}")

        if not np.isfinite(loss):
            raise RuntimeError("Loss exploded/NaN. Lower lr (e.g., 0.05 or 0.01).")

    return W

W = fit_logreg_L1(Xtr, y_train, lr=0.1, epochs=2000, lam=1e-3, seed=450)

def predict_proba(X, W):
    return sigmoid(X @ W)

def predict_label(X, W, thresh=0.5):
    return (predict_proba(X, W) >= thresh).astype(int)

proba = predict_proba(Xte, W)
pred  = predict_label(Xte, W)

# metrics
tp = int(((pred == 1) & (y_test == 1)).sum())
tn = int(((pred == 0) & (y_test == 0)).sum())
fp = int(((pred == 1) & (y_test == 0)).sum())
fn = int(((pred == 0) & (y_test == 1)).sum())

acc  = (tp + tn) / (tp + tn + fp + fn) #% of true negatives and positives out of all the values
prec = tp / (tp + fp) if (tp + fp) else 0.0 #Number of true positives in all the true and false positives.
rec  = tp / (tp + fn) if (tp + fn) else 0.0 #%of true positives compared to all true positives and false negatives (All real positives)
f1   = 2 * prec * rec / (prec + rec) if (prec + rec) else 0.0

print(f"\nPositives [{tp:3}  {fp:3}]")
print(f"Negatives [{tn:3}  {fn:3}]")
print(f"\nTest Accuracy={acc:.3f} Recall={rec:.3f}  Precision={prec:.3f}  F1={f1:.3f}")

# inspect learned weights
feat_names = ["(bias)", "Variance", "Skewness", "Curtosis", "Entropy"]
for name, w in zip(feat_names, W):
    print(f"{name:>10s}: {w:+.4f}")

classes: {0: 762, 1: 610}
unique: [0 1]
      Variance  Skewness  Curtosis  Entropy  Class
0      3.62160   8.66610   -2.8073 -0.44699      0
1      4.54590   8.16740   -2.4586 -1.46210      0
2      3.86600  -2.63830    1.9242  0.10645      0
3      3.45660   9.52280   -4.0112 -3.59440      0
4      0.32924  -4.45520    4.5718 -0.98880      0
...        ...       ...       ...      ...    ...
1367   0.40614   1.34920   -1.4501 -0.55949      1
1368  -1.38870  -4.87730    6.4774  0.34179      1
1369  -3.75030 -13.45860   17.5932 -2.77710      1
1370  -3.56370  -8.38270   12.3930 -1.28230      1
1371  -2.54190  -0.65804    2.6842  1.19520      1

[1372 rows x 5 columns]
epoch    0  logloss=0.6940  + L1=0.6940  nnz=4
epoch  200  logloss=0.1985  + L1=0.2029  nnz=4
epoch  400  logloss=0.1340  + L1=0.1404  nnz=4
epoch  600  logloss=0.1068  + L1=0.1144  nnz=4
epoch  800  logloss=0.0918  + L1=0.1003  nnz=4
epoch 1000  logloss=0.0822  + L1=0.0915  nnz=4
epoch 1200  logloss=0.0755  + L1=0.0854  

#Logistic regression with L2

In [ ]:
#Import libraries
import pandas as pd
import numpy as np

#Open Dataset
url = "https://raw.githubusercontent.com/Youssef-ElBakry/Logistic-Regression-ML/main/data_banknote_authentication.csv"
cols = ["Variance", "Skewness", "Curtosis", "Entropy", "Class"] #Adding headers to file as dataset has no headers
df = pd.read_csv(url, header=None, names=cols)
df["Class"] = df["Class"].astype(int)
print("classes:", df["Class"].value_counts().to_dict())
print("unique:", df["Class"].unique())
print(df)

#Define global values
trainingRatio = 0.8
LabelCol = "LABEL"
seed = 450 #Fixed seed for rng as it makes results reproducable


#Split data into training and testing
#80/20 used as there is no hyperparameter tuning and therefore no need for validation
rng = np.random.default_rng(seed)
test_I = []
train_I = []

#Shuffle and sort data entries into training and testing
for key, row in df.groupby("Class", sort="False"):
  index = row.index.to_numpy()
  rng.shuffle(index)
  trainingRows = int(round(len(index) * trainingRatio)) #Determine size of training set in each class. This allows for a more even split. (80% of the Authentic class and 80% of the forged class)
  train_I.extend(index[:trainingRows])
  test_I.extend(index[trainingRows:])

#Shuffle training and testing set again as they are currently grouped by class
train = df.loc[train_I].sample(frac=1, random_state=seed).reset_index(drop=True)
test = df.loc[test_I].sample(frac=1, random_state=seed).reset_index(drop=True)

FEATS = ["Variance", "Skewness", "Curtosis", "Entropy"]

X_train = train[FEATS].to_numpy(dtype=np.float64)
y_train = train["Class"].to_numpy(dtype=np.float64)
X_test  = test[FEATS].to_numpy(dtype=np.float64)
y_test  = test["Class"].to_numpy(dtype=np.float64)

#Standardise data using mean and standardisation to find z-scores
mu = X_train.mean(axis=0) #Calculate mean of each column
sd = X_train.std(axis=0) #Calculate standard deviation for each column

X_train_std = (X_train - mu) / sd #Turn each value in each column to a zscore
X_test_std = (X_test - mu) / sd

#Add a column of ones, to allow for intercept
def add_bias(X):
  return np.c_[np.ones((X.shape[0], 1)), X]  # shape: (N, D+1)

Xtr = add_bias(X_train_std)   # (n_train, 1+4)
Xte = add_bias(X_test_std)

#Sigmoid function
def sigmoid(z):
  #Cuts off z so that it's value its >500 or <-500 as those numbers to the power of e is an unnecessarily large number
  z = np.clip(z, -500, 500)
  return 1.0 / (1.0 + np.exp(-z))

def logloss_L2_and_grad(W, X, y,Lam):
  #Get number of features in array
  N = X.shape[0]

  z = X @ W   #Matrix multiplication of array and weights
  p = sigmoid(z)    #Map each score to a point in the sigmoid graph

  # log loss (cross-entropy)
  eps = 1e-12   #avoid log(0) by setting adding a very small number
  loss = -np.mean(y*np.log(p + eps) + (1 - y)*np.log(1 - p + eps))

  # L2 penalty (exclude bias W[0])
  l2 = 0.5 * Lam * np.sum(W[1:]**2)

  err  = p - y
  grad = (X.T @ err) / N
  # add L2 gradient on non-bias weights
  g = grad.copy()
  g[1:] += Lam * W[1:]
  return loss + l2, g

def fit_logreg(X, y, lr=0.1, epochs=2000, Lam=1e-3, seed=450):
    rng = np.random.default_rng(seed)
    W = rng.normal(scale=0.01, size=X.shape[1])   # small random init, shape (D,)
    for t in range(epochs):
        loss, grad = logloss_L2_and_grad(W, X, y,Lam)
        W -= lr * grad #Gradient decent
        if (t % 200 == 0 or t == epochs - 1):
            print(f"epoch {t:4d}  loss={loss:.4f}")
        if not np.isfinite(loss):
            raise RuntimeError("Loss exploded/NaN. Lower lr (e.g., 0.05 or 0.01).")
    return W

W = fit_logreg(Xtr, y_train, lr=0.1, epochs=2000, Lam=1e-3, seed=450)

def predict_proba(X, W):
    return sigmoid(X @ W)

def predict_label(X, W, thresh=0.5):
    return (predict_proba(X, W) >= thresh).astype(int)

proba = predict_proba(Xte, W)
pred  = predict_label(Xte, W)

# metrics
tp = int(((pred == 1) & (y_test == 1)).sum())
tn = int(((pred == 0) & (y_test == 0)).sum())
fp = int(((pred == 1) & (y_test == 0)).sum())
fn = int(((pred == 0) & (y_test == 1)).sum())

acc  = (tp + tn) / (tp + tn + fp + fn) #% of true negatives and positives out of all the values
prec = tp / (tp + fp) if (tp + fp) else 0.0 #Number of true positives in all the true and false positives.
rec  = tp / (tp + fn) if (tp + fn) else 0.0 #%of true positives compared to all true positives and false negatives (All real positives)
f1   = 2 * prec * rec / (prec + rec) if (prec + rec) else 0.0

print(f"\nPositives [{tp:3}  {fp:3}]")
print(f"Negatives [{tn:3}  {fn:3}]")
print(f"\nTest Accuracy={acc:.3f} Recall={rec:.3f}  Precision={prec:.3f}  F1={f1:.3f}")

# inspect learned weights
feat_names = ["(bias)", "Variance", "Skewness", "Curtosis", "Entropy"]
for name, w in zip(feat_names, W):
    print(f"{name:>10s}: {w:+.4f}")

classes: {0: 762, 1: 610}
unique: [0 1]
      Variance  Skewness  Curtosis  Entropy  Class
0      3.62160   8.66610   -2.8073 -0.44699      0
1      4.54590   8.16740   -2.4586 -1.46210      0
2      3.86600  -2.63830    1.9242  0.10645      0
3      3.45660   9.52280   -4.0112 -3.59440      0
4      0.32924  -4.45520    4.5718 -0.98880      0
...        ...       ...       ...      ...    ...
1367   0.40614   1.34920   -1.4501 -0.55949      1
1368  -1.38870  -4.87730    6.4774  0.34179      1
1369  -3.75030 -13.45860   17.5932 -2.77710      1
1370  -3.56370  -8.38270   12.3930 -1.28230      1
1371  -2.54190  -0.65804    2.6842  1.19520      1

[1372 rows x 5 columns]
epoch    0  loss=0.6940
epoch  200  loss=0.2020
epoch  400  loss=0.1416
epoch  600  loss=0.1178
epoch  800  loss=0.1055
epoch 1000  loss=0.0982
epoch 1200  loss=0.0935
epoch 1400  loss=0.0902
epoch 1600  loss=0.0879
epoch 1800  loss=0.0862
epoch 1999  loss=0.0849
0.976

Positives [122    6]
Negatives [146    0]

Test Accu

#Logistic regression with L2 & Lambda tuning

In [ ]:
#Import libraries
import pandas as pd
import numpy as np

#Open Dataset
url = "https://raw.githubusercontent.com/Youssef-ElBakry/Logistic-Regression-ML/main/data_banknote_authentication.csv"
cols = ["Variance", "Skewness", "Curtosis", "Entropy", "Class"] #Adding headers to file as dataset has no headers
df = pd.read_csv(url, header=None, names=cols)
df["Class"] = df["Class"].astype(int)
print("classes:", df["Class"].value_counts().to_dict())
print("unique:", df["Class"].unique())
print(df)

#Define global values
trainingRatio = 0.8
LabelCol = "LABEL"
seed = 450 #Fixed seed for rng as it makes results reproducable


#Split data into training and testing
#80/20 used as there is no hyperparameter tuning and therefore no need for validation
rng = np.random.default_rng(seed)
test_I = []
train_I = []

#Shuffle and sort data entries into training and testing
for key, row in df.groupby("Class", sort="False"):
  index = row.index.to_numpy()
  rng.shuffle(index)
  trainingRows = int(round(len(index) * trainingRatio)) #Determine size of training set in each class. This allows for a more even split. (80% of the Authentic class and 80% of the forged class)
  train_I.extend(index[:trainingRows])
  test_I.extend(index[trainingRows:])

#Shuffle training and testing set again as they are currently grouped by class
train = df.loc[train_I].sample(frac=1, random_state=seed).reset_index(drop=True)
test = df.loc[test_I].sample(frac=1, random_state=seed).reset_index(drop=True)

FEATS = ["Variance", "Skewness", "Curtosis", "Entropy"]

X_train = train[FEATS].to_numpy(dtype=np.float64)
y_train = train["Class"].to_numpy(dtype=np.float64)
X_test  = test[FEATS].to_numpy(dtype=np.float64)
y_test  = test["Class"].to_numpy(dtype=np.float64)

#Standardise data using mean and standardisation to find z-scores
mu = X_train.mean(axis=0) #Calculate mean of each column
sd = X_train.std(axis=0) #Calculate standard deviation for each column

X_train_std = (X_train - mu) / sd #Turn each value in each column to a zscore
X_test_std = (X_test - mu) / sd

#Add a column of ones, to allow for intercept
def add_bias(X):
  return np.c_[np.ones((X.shape[0], 1)), X]  # shape: (N, D+1)

Xtr = add_bias(X_train_std)   # (n_train, 1+4)
Xte = add_bias(X_test_std)

#Sigmoid function
def sigmoid(z):
  #Cuts off z so that it's value its >500 or <-500 as those numbers to the power of e is an unnecessarily large number
  z = np.clip(z, -500, 500)
  return 1.0 / (1.0 + np.exp(-z))

def logloss_L2_and_grad(W, X, y,Lam):
  #Get number of features in array
  N = X.shape[0]

  z = X @ W   #Matrix multiplication of array and weights
  p = sigmoid(z)    #Map each score to a point in the sigmoid graph

  # log loss (cross-entropy)
  eps = 1e-12   #avoid log(0) by setting adding a very small number
  loss = -np.mean(y*np.log(p + eps) + (1 - y)*np.log(1 - p + eps))

  # L2 penalty (exclude bias W[0])
  l2 = 0.5 * Lam * np.sum(W[1:]**2)

  err  = p - y
  grad = (X.T @ err) / N
  # add L2 gradient on non-bias weights
  g = grad.copy()
  g[1:] += Lam * W[1:]
  return loss + l2, g

def fit_logreg(X, y, seed, Lam, lr=0.1, epochs=2000):
    rng = np.random.default_rng(seed)
    W = rng.normal(scale=0.01, size=X.shape[1])   # small random init, shape (D,)
    for t in range(epochs):
        loss, grad = logloss_L2_and_grad(W, X, y,Lam)
        W -= lr * grad #Gradient decent
        if not np.isfinite(loss):
            raise RuntimeError("Loss exploded/NaN. Lower lr (e.g., 0.05 or 0.01).")
    return W


def predict_proba(X, W):
    return sigmoid(X @ W)

def predict_label(X, W, thresh=0.5):
    return (predict_proba(X, W) >= thresh).astype(int)

Lam_grid = [0, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
bestF1 = bestLam = bestAcc = bestPrec = bestRec = bestW = 0
for Lam in Lam_grid:
  W = fit_logreg(Xtr, y_train, seed, Lam, lr=0.1, epochs=2000)
  proba = predict_proba(Xte, W)
  pred  = predict_label(Xte, W)
  tp = int(((pred == 1) & (y_test == 1)).sum())
  tn = int(((pred == 0) & (y_test == 0)).sum())
  fp = int(((pred == 1) & (y_test == 0)).sum())
  fn = int(((pred == 0) & (y_test == 1)).sum())
  acc  = (tp + tn) / (tp + tn + fp + fn)
  prec = tp / (tp + fp) if (tp + fp) else 0.0
  rec  = tp / (tp + fn) if (tp + fn) else 0.0
  f1   = 2 * prec * rec / (prec + rec) if (prec + rec) else 0.0
  print(f1)
  if f1 > bestF1:
    bestLam = Lam
    bestPrec = prec
    bestAcc = acc
    bestRec = rec
    bestF1 = f1
    bestW = W



print(f"\nPositives [{tp:3}  {fp:3}]")
print(f"Negatives [{tn:3}  {fn:3}]")
print(Lam)
print(f"\nTest Accuracy={bestAcc:.3f} Recall={bestRec:.3f}  Precision={bestPrec:.3f}  F1={bestF1:.3f}")

# inspect learned weights
feat_names = ["(bias)", "Variance", "Skewness", "Curtosis", "Entropy"]
for name, w in zip(feat_names, W):
    print(f"{name:>10s}: {w:+.4f}")

classes: {0: 762, 1: 610}
unique: [0 1]
      Variance  Skewness  Curtosis  Entropy  Class
0      3.62160   8.66610   -2.8073 -0.44699      0
1      4.54590   8.16740   -2.4586 -1.46210      0
2      3.86600  -2.63830    1.9242  0.10645      0
3      3.45660   9.52280   -4.0112 -3.59440      0
4      0.32924  -4.45520    4.5718 -0.98880      0
...        ...       ...       ...      ...    ...
1367   0.40614   1.34920   -1.4501 -0.55949      1
1368  -1.38870  -4.87730    6.4774  0.34179      1
1369  -3.75030 -13.45860   17.5932 -2.77710      1
1370  -3.56370  -8.38270   12.3930 -1.28230      1
1371  -2.54190  -0.65804    2.6842  1.19520      1

[1372 rows x 5 columns]
0.9799196787148594
0.9344262295081968
0.9641434262948207
0.976
0.9799196787148594
0.9799196787148594

Positives [122    5]
Negatives [147    0]
1e-05

Test Accuracy=0.982 Recall=1.000  Precision=0.961  F1=0.980
    (bias): -1.0168
  Variance: -4.2070
  Skewness: -3.8517
  Curtosis: -3.6155
   Entropy: +0.2051


#Final results
Basic Logistic regression
  - Accuracy:0.982
  - Precision:0.961
  - Recall:1.000
  - F1:0.980

Logistic regression W/ L1
  - Accuracy: 0.978
  - Precision:0.953
  - Recall: 1.000
  - F1: 0.976

Logistic regression W/ L2
  - Accuracy: 0.978
  - Precision: 0.953  
  - Recall: 1.000
  - F1: 0.976

Logistic regression W/ L2 & Lamda tuning
  - Accuracy: 0.982
  - Precision: 0.961
  - Recall: 1.000
  - F1: 0.980
